In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# choose default setting or AMD setting

In [3]:
# default setting
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models

In [3]:
# AMD setting
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ["PLAIDML_EXPERIMENTAL"] = "1"
os.environ["PLAIDML_DEVICE_IDS"] = "opencl_amd_ellesmere.0"

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import layers, models

Using plaidml.keras.backend backend.


In [4]:
# start after setting

In [7]:
# you can change this values
class_names = ['Non', 'Cut', 'Dissolve', 'Fade']
batch_size = 10
train_steps = 40
valid_steps = 10
epochs = 5

In [8]:
# 
train_image_generator = ImageDataGenerator(rescale=1./255)
valid_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='train',
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='binary')
valid_data_gen = valid_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='valid',
                                                           target_size=(224, 224),
                                                           class_mode='binary')

Found 8000 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.


In [9]:
vgg16_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
#vgg16_model.summary()
vgg16_model.trainable = False
model = models.Sequential()
model.add(vgg16_model)
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
#model.summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=valid_data_gen,
    validation_steps=valid_steps
)

Epoch 1/5
40/40 [==============================] - 71s 2s/step - loss: 6.1134 - acc: 0.4650 - val_loss: 0.6377 - val_acc: 0.7600
Epoch 2/5
40/40 [==============================] - 57s 1s/step - loss: 0.8521 - acc: 0.6575 - val_loss: 0.7393 - val_acc: 0.7200
Epoch 3/5
40/40 [==============================] - 57s 1s/step - loss: 0.7524 - acc: 0.7025 - val_loss: 0.7656 - val_acc: 0.7000
Epoch 4/5
40/40 [==============================] - 56s 1s/step - loss: 0.7966 - acc: 0.6875 - val_loss: 0.6873 - val_acc: 0.7100
Epoch 5/5
40/40 [==============================] - 57s 1s/step - loss: 0.6140 - acc: 0.7400 - val_loss: 0.8020 - val_acc: 0.7500
